In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import json
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from sklearn.externals import joblib
from keras.utils import to_categorical

from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.models import Sequential, load_model, model_from_json






Using TensorFlow backend.


In [2]:
# with open('X_train.jlib', 'wb') as file:
#     joblib.dump(X_train, file)
#     print(X_train.shape)
# with open('X_valid.jlib', 'wb') as file:
#     joblib.dump(X_valid, file)
#     print(X_valid.shape)
# with open('y_train', 'wb') as file:
#     joblib.dump(y_train, file)
#     print(y_train.shape)
# with open('y_valid.jlib', 'wb') as file:
#     joblib.dump(y_valid, file)
#     print(y_valid.shape)
# X_train= None
# X_valid= None
# y_train= None
# y_valid = None
# with open('X_train.jlib', 'rb') as file:
#     X_train = joblib.load( file)
#     print(X_train.shape)
# with open('X_valid.jlib', 'rb') as file:
#     X_valid = joblib.load(file)
#     print(X_valid.shape)
# with open('y_train.jlib', 'rb') as file:
#     y_train = joblib.load( file)
#     print(y_train.shape)
# with open('y_valid.jlib', 'rb') as file:
#     y_valid = joblib.load( file)
#     print(y_valid.shape)


24232

In [14]:
TRAIN_LABEL_PATH = "MLDS_hw2_data/training_label.json"


    
def read_data(path):
    with open(path) as data_file:    
        y_data = json.load(data_file)
    
    videoId = []
    videoSeq = []

    for y in y_data:
        for idx, cap in enumerate(y['caption']):
            cap = "<bos> " + cap + " <eos>"
            videoId.append(y['id'])
            videoSeq.append(cap)
            
       
    return videoId, videoSeq


def getVId(path):
    TRAIN_LABEL_PATH = "MLDS_hw2_data/training_label.json"
    
    videoId, videoSeq = read_data(path)
    tokenize(videoId, videoSeq)
    curFilename = videoId[0]
    vCount = 0
    y_videoId = []
    for idx, seq in  enumerate(videoId):
            if(videoId[idx] == curFilename):
                vCount = vCount + 1
                if(vCount > 2):
                    continue
            else:
                vCount = 1
                curFilename = videoId[idx]
            y_videoId.append(videoId[idx])
    return videoId

def tokenize(videoId, videoSeq):
    MAX_WORDS = 1024
    tokenizer = Tokenizer(num_words=MAX_WORDS)

    tokenizer.fit_on_texts(videoSeq)
    word_index = tokenizer.word_index
#     print(word_index)
   
    
    print ('Convert to index sequences.')
    train_sequences = tokenizer.texts_to_sequences(videoSeq)
    train_sequences = np.array(train_sequences)
    print(train_sequences[0])

    train_sequences = pad_sequences(train_sequences)
    print(train_sequences.shape)
    max_seq_length = train_sequences.shape[1]
    print(max_seq_length)

    y_data = []
    y_videoId= []
    curFilename = videoId[0]
    vCount = 0
    for idx, seq in  enumerate(train_sequences):
        if(videoId[idx] == curFilename):
            vCount = vCount + 1
            if(vCount > 2):
                continue
        else:
            vCount = 1
            curFilename = videoId[idx]
        y = to_categorical(seq, MAX_WORDS)

        y_data.append(y)
        y_videoId.append(videoId[idx])

    y_data = np.array(y_data)   
    print(y_data.shape)
#     with open('data/y_data1024.jlib', 'wb') as file:
#         joblib.dump(y_data, file)
#     with open('data/tokinzer1024', 'wb') as file:
#         joblib.dump(tokenizer, file)
#     genX_data()
  
  
    return y_videoId, y_data, tokenizer

def genX_data():
    TRAIN_FEATURE_DIR = "./MLDS_hw2_data/training_data/feat/"
    x_data = {}
    for filename in os.listdir(TRAIN_FEATURE_DIR):
        f = np.load(TRAIN_FEATURE_DIR + filename)
        x_data[filename[:-4]] = f
        X_data = []
    for vid in y_videoId:
        X_data.append(x_data[vid])
    X_data = np.array(X_data)
    with open('x_data2.jlib', 'wb') as file:
        joblib.dump(X_data, file)


# videoId, videoSeq = read_data(TRAIN_LABEL_PATH)
# y_videoId, y_data, tokenizer= tokenize(videoId, videoSeq)

def getWords(word_index, seq):
    for word in seq:
        idx = np.argmax(word)
        if idx == 0:
            continue
        print(list(word_index.keys())[list(word_index.values()).index(idx)])
        
# print(list(mydict.keys())[list(mydict.values()).index(16)]) # Prints george
    


getWords(tokinzer.word_index, decoder_target_data[0])



bos
a
woman
goes
under
a
horse
eos


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   2
   1   7 725 451   1  53   3]


array([ 0.,  1.,  0., ...,  0.,  0.,  0.])

In [2]:
# load data
encoder_input_data = None
decoder_data = None
tokinzer = None
with open('data/X_data2.jlib', 'rb') as file:
    encoder_input_data = joblib.load(file)
    print(encoder_input_data.shape)
with open('data/y_data1024.jlib', 'rb') as file:
    decoder_data = joblib.load(file)
    print(decoder_data.shape)
with open('data/tokinzer1024', 'rb') as file:
    tokinzer = joblib.load(file)
    print(len(tokinzer.word_index))

(2900, 80, 4096)
(2900, 36, 1024)
6018


In [3]:
decoder_input_data = []
decoder_target_data = []
for e in decoder_data:
    i = e[:-1]
    o = e[1:]
    decoder_input_data.append(i)
    decoder_target_data.append(o)
decoder_input_data = np.array(decoder_input_data)
decoder_target_data = np.array(decoder_target_data)



In [4]:

num_encoder_tokens = encoder_input_data.shape[2]
num_decoder_tokens = decoder_input_data.shape[2]
latent_dim = 512
batch_size = 200
epochs = 50


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)

encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

# Run training
model.compile(metrics=['accuracy'], optimizer='adam', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.15)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, 4096)    0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, None, 1024)    0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                    [(None, 512), (None,  9439232     input_1[0][0]                    
____________________________________________________________________________________________________
lstm_2 (LSTM)                    [(None, None, 512), ( 3147776     input_2[0][0]                    
                                                                   lstm_1[0][1]            

In [13]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [5]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
encoder_model.summary()
decoder_model.summary()


# decode_sequence(input_seq, encoder_model,decoder_model )
encoder_model.save('encoder_model.h5')
decoder_model.save_weights('decoder_model_weights.h5')
encoder_input_data.shape



In [8]:
def decode_model():
    decoder_inputs = Input(shape=(None, num_decoder_tokens))
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    return decoder_model

def decode_sequence(input_seq, encoder_model, decoder_model):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, tokenizer.word_index['bos']] = 1

    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0:
            sampled_char = ''
        else:
            sampled_char = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(sampled_token_index)]
        decoded_sentence.append(sampled_char)

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'eos' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence
num_encoder_tokens = 4096
num_decoder_tokens = 1024
latent_dim = 512

enc_model = load_model('encoder_model.h5')
enc_model.summary()
dec_model = decode_model()
dec_model.load_weights('decoder_model_weights.h5')
# dec_model.summary()
dec_model.summary()



/home/alvin/miniconda3/lib/python3.6/site-packages/keras/models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 4096)        0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 512), (None, 512) 9439232   
Total params: 9,439,232
Trainable params: 9,439,232
Non-trainable params: 0
_________________________________________________________________
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, None, 1024)    0                                            
____________________________________________________________________________________________________
input_8 (InputLayer)             (None, 512)           0                                            
_______________________

In [9]:
TRAIN_LABEL_PATH = "MLDS_hw2_data/training_label.json"
TEST_LABEL_PATH = "MLDS_hw2_data/testing_label.json"
TEST_FEATURE_DIR = "./MLDS_hw2_data/testing_data/feat/"

tokenizer = None
with open('data/tokinzer1024', 'rb') as file:
    tokenizer = joblib.load(file)
    print(len(tokenizer.word_index))

    
def read_data(path):
    with open(path) as data_file:    
        y_data = json.load(data_file)
    
    videoId = []
    videoSeq = []

    for y in y_data:
        for idx, cap in enumerate(y['caption']):
            cap = "<bos> " + cap + " <eos>"
            videoId.append(y['id'])
            videoSeq.append(cap)
            
       
    return videoId, videoSeq


def getVId(path): 
    videoId, videoSeq = read_data(path)
    tokenize(videoId, videoSeq)
    curFilename = videoId[0]
    vCount = 0
    y_videoId = []
    for idx, seq in  enumerate(videoId):
            if(videoId[idx] == curFilename):
                vCount = vCount + 1
                if(vCount > 2):
                    continue
            else:
                vCount = 1
                curFilename = videoId[idx]
            y_videoId.append(videoId[idx])
    return videoId

def tokenize(videoId, videoSeq, tokenizer):
    MAX_WORDS = 1024
    if(tokenizer == None):
        tokenizer = Tokenizer(num_words=MAX_WORDS)
        tokenizer.fit_on_texts(videoSeq)
        
    word_index = tokenizer.word_index
#     print(word_index)
   
    
    print ('Convert to index sequences.')
    train_sequences = tokenizer.texts_to_sequences(videoSeq)
    train_sequences = np.array(train_sequences)

    train_sequences = pad_sequences(train_sequences, maxlen= 36)
    print(train_sequences.shape)
    max_seq_length = train_sequences.shape[1]
    print(max_seq_length)

    y_data = []
    y_videoId= []
    curFilename = videoId[0]
    vCount = 0
    for idx, seq in  enumerate(train_sequences):
        if(videoId[idx] == curFilename):
            vCount = vCount + 1
            if(vCount > 2):
                continue
        else:
            vCount = 1
            curFilename = videoId[idx]
        y = to_categorical(seq, MAX_WORDS)

        y_data.append(y)
        y_videoId.append(videoId[idx])

    y_data = np.array(y_data)   
    print(y_data.shape)
#     with open('data/y_data1024.jlib', 'wb') as file:
#         joblib.dump(y_data, file)
#     with open('data/tokinzer1024', 'wb') as file:
#         joblib.dump(tokenizer, file)
#     genX_data()
  
  
    return y_videoId, y_data, tokenizer

def genX_test_data(path):   
    X_test = []
    X_test_filename = []
    for filename in os.listdir(path):
        f = np.load(path + filename)
        X_test.append(f)
        X_test_filename.append(filename[:-4])
    X_test = np.array(X_test)
    return X_test, X_test_filename
        
       




def getWords(word_index, seq):
    for word in seq:
        idx = np.argmax(word)
        if idx == 0:
            continue
        print(list(word_index.keys())[list(word_index.values()).index(idx)])



X_test, X_test_filename = genX_test_data(TEST_FEATURE_DIR)
X_test.shape
# test_videoId, test_videoSeq = read_data(TEST_LABEL_PATH)
# test_videoId, y_test, tokenizer= tokenize(videoId, videoSeq, tokenizer)
# print(list(mydict.keys())[list(mydict.values()).index(16)]) # Prints george

with open('result.csv', 'w') as file:
    for idx, x in enumerate(X_test): 
        decoded_sentence = decode_sequence(x.reshape(-1, 80, 4096), enc_model,dec_model)
        decode_str = ''
        filter_string = ['bos', 'eos']
        for c in decoded_sentence:
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str += c + ' '
        print(X_test_filename[idx] + ' > ' + decode_str)
        file.write(X_test_filename[idx] + ',' + decode_str + '\n')


6018
04Gt01vatkk_248_265.avi > a man is slicing a 
04Gt01vatkk_308_321.avi > a man is cutting a 
0lh_UWF9ZP4_27_31.avi > a person is mixing a bowl 
0lh_UWF9ZP4_62_69.avi > a person is cooking meat 
1Sp2__RCT0c_11_15.avi > a man is riding a horse in 
30GeJHYoerk_121_126.avi > a man is playing a guitar 
3qqEKTPxLNs_1_15.avi > a man is 
4PcL6-mjRNk_11_18.avi > a man is a a box 
4xVGpDmA4lE_23_33.avi > a man is running in 
5HAf_INrFy0_3_25.avi > a cat is walking 
5YJaS2Eswg0_22_26.avi > a man is playing a guitar 
6JnGBs88sL0_4_10.avi > a man is riding a horse 
6q1dX6thX3E_286_295.avi > a man is playing 
71soiLO6I9U_15_24.avi > a person is cutting 
HV12kTtdTT4_5_14.avi > a man is playing a guitar 
IhwPQL9dFYc_124_129.avi > a person is slicing a 
inzk2fTUe1w_1_15.avi > a man is a bread 
J---aiyznGQ_0_6.avi > a man is playing a guitar 
j2Dhf-xFUxU_13_20.avi > a person is cutting 
Jag7oTemldY_12_25.avi > a man is walking 
jbzaMtPYtl8_48_58.avi > a man is playing a guitar 
Je3V7U5Ctj4_569_576.a